# Graph Neural Networks (GNNs)

From MLPs to GCNs and GATs.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from tqdm import tqdm

In [2]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The Cora dataset is a benchmark dataset for graph neural networks. The dataset contains data about 2708 scientific publications. These publications are the nodes of the graph. An edge between nodes (publications) is created when a publication references the other one. The target is to predict the subject of each paper, there are seven classes in total.

In [ ]:
!pip install torch-geometric
from torch_geometric.datasets import Planetoid

dataset= Planetoid(root='.', name='Cora', force_reload=True)
data= dataset[0]

In [4]:
data_size= data.x.shape[0]
dev_size= 500
test_size= 500
train_size= data_size - dev_size - test_size

train_mask= torch.tensor([i< train_size for i in range(data_size)])
dev_mask= torch.tensor([i>= train_size and i< (data_size - test_size) for i in range(data_size)])
test_mask= torch.tensor([i>= (train_size + dev_size) for i in range(data_size)])

data.train_mask= train_mask
data.val_mask= dev_mask
data.test_mask= test_mask

In [5]:
data= data.to(device)

Xtr, Ytr= data.x[data.train_mask], data.y[data.train_mask]
Xdev, Ydev= data.x[data.val_mask], data.y[data.val_mask]
Xte, Yte= data.x[data.test_mask], data.y[data.test_mask]

edge_idx= data.edge_index

num_inputs= data.x.shape[1]                # used for input_dim
num_labels= len(set(data.y.cpu().numpy())) # used for output_dim

num_inputs, num_labels

(1433, 7)

# Neural Network - MLP

In [6]:
class MLPLayer(nn.Module):
    """
    Implements one customizable CNN layer.
    Input -> Linear -> Norm -> ReLU -> Dropout -> Output
    """

    def __init__(self, input_dim, output_dim, layer_norm=True, activation=None,
                 dropout=0.0) -> None:
        super(MLPLayer, self).__init__()
        # Fully connected (FC) layer
        self.fc_layer= nn.Linear(input_dim, output_dim)
        # Normalization
        self.norm= nn.LayerNorm(output_dim) if layer_norm else None
        # Activation function
        self.activ= activation
        # Dropout layer for regularization
        self.dropout= nn.Dropout(p=dropout) if dropout> 0.0 else None


    def forward(self, x):
        x= self.fc_layer(x)
        if self.norm is not None:
            x= self.norm(x)
        if self.activ is not None:
            x= self.activ(x)
        if self.dropout is not None:
            x= self.dropout(x)

        return x



class MLP(nn.Module):
    """
    Implements a customizable MLP.
    nn.ReLU(inplace=True) is the default activation function.
    """

    def __init__(self, input_dim, hidden_dim=[16,], output_dim=1, layer_norm=False,
                 activation=None, dropout=0.0) -> None:
        super(MLP, self).__init__()
        # Activation function -- ReLU is the default
        activation= nn.ReLU(inplace=True) if activation is None else activation

        if isinstance(hidden_dim, int):
            hidden_dim= [hidden_dim]
        n_hidden_layers= len(hidden_dim)

        if n_hidden_layers== 0:
            raise Exception('hidden_dim cannot be an empty list')

        self.fc_in= MLPLayer(input_dim, hidden_dim[0], layer_norm, activation, dropout)

        if n_hidden_layers> 1:
            self.fc_hn= nn.Sequential(*[
                MLPLayer(d, hidden_dim[i+1], layer_norm, activation, dropout)
                for i, d in enumerate(hidden_dim[:-1])
            ])
        else: self.fc_hn= None

        self.fc_out= MLPLayer(hidden_dim[-1], output_dim, layer_norm=False)


    def forward(self, x):  # no graph structure, only node features
        x= self.fc_in(x)
        if self.fc_hn is not None:
            x= self.fc_hn(x)
        x= self.fc_out(x)

        return F.log_softmax(x, dim=1)


In [7]:
model= MLP(input_dim=num_inputs, hidden_dim=[32,], output_dim=num_labels,
           layer_norm=True, dropout=0.1).to(device)

total_params= sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of parameters: {total_params}')

Number of parameters: 46183


# Graph Convolutional Network - GCN

There are three common types of prediction tasks in graphs:
- We can predict on graph level. The input of the model is many different graphs, and every graph gets one classification. For example the class a molecule belongs to: every molecule is represented by one graph, and every molecule needs a prediction. Another example is image classification. Yes, images can also be represented as graphs!
- Another way to use GNNs is by predicting on node level. The input of the GNN is one graph, and every node needs a prediction. This prediction is a characteristic of the node. Node regression is of course possible as well. Compared to classification, we only need to change the output layer activation function, the loss function, evaluation metric, and obviously the target.
- Finally, we can predict on edge level. The value of an edge is predicted, or the likelihood of an edge that will appear soon. An example is recommended friends on social media (a.k.a. link prediction).

For understanding one node, we need to look at its neighborhood and include that information in the GCN. The idea of graph convolution is to allow each node to pass messages to neighboring nodes, such that the content of each node has the ability to interact with neighboring nodes. This happens in an iterative loop, which can be conceptualized as a "layer" of the model. The modified messages from one layer can be passed to another layer for further message passing. This can be done in as many layers as we desire. The number of layers we have dictates how far into the distance a particular node can "see".

Just like how, in a traditional convolutional network, we can stack layers of convolutions on top of each other to create more in-depth understanding; in graph convolutional networks, we can stack layers of these neural networks on top of one another which modify how signals are passed.

# Implementing a GCNConv module from scratch

A Graph Convolutional Network (https://arxiv.org/abs/1609.02907) is described by:

$$
H^{(l+1)} = \sigma (\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}H^{(l)} W^{(l)}).
$$

Here $H$ represents the "hidden state" of each node in our graph, i.e. the vector associated with each node. We are creating a new hidden state for our nodes ($H^{(l+1)}$, $l+1$ symbolizing the next layer) based on some math and the previous hidden state ($H^{(l)}$, $l$ symbolizing the current layer). The hidden state, in this example, is formatted as a matrix, where each row of the matrix corresponds with a particular node.

Our input hidden state is multiplied by a weight matrix and then passed through an activation function. The way matrix multiplication shakes out, this is equivalent to passing our data through a neural network where the parameters of that neural network are within the matrix $W$.

In [8]:
class GCNConv(nn.Module):
    """
    Implements a GCNConv module.
    """

    def __init__(self, input_dim, output_dim) -> None:
        super(GCNConv, self).__init__()
        # Learnable linear transformation
        self.W= nn.Linear(input_dim, output_dim)


    def forward(self, x, edge_index):
        num_nodes= x.size(0)
        device= x.device

        # create the adjacency matrix (including self-loops)
        self_loops= torch.arange(num_nodes, device=device).unsqueeze(0).repeat(2, 1)
        edge_index= torch.cat([edge_index, self_loops], dim=1)
        # extract source (row) and destination (col) indices from edge_index
        row, col= edge_index

        # construct the adjacency matrix
        A= torch.zeros((num_nodes, num_nodes), dtype=x.dtype, device=device)
        A[row, col]= 1  # fill the adjacency matrix

        # compute degree matrix
        deg= torch.bincount(row, minlength=num_nodes).float()
        # compute D^(-1/2) avoiding division by zero
        deg_inv_sqrt= deg.pow(-0.5)
        deg_inv_sqrt[deg == 0]= 0

        # calculate D^(-1/2) * A * D^(-1/2)
        D_inv_sqrt= torch.diag(deg_inv_sqrt)
        A_norm= D_inv_sqrt @ A @ D_inv_sqrt  # normalized A

        # propagate node features through the normalized adjacency matrix
        x= A_norm @ x

        return self.W(x)  # apply the learned transformation


There is one important step we should take before actually implementing a GCN, and that is normalization. Imagine, without normalization, nodes with more connections (e.g. one node having 10 neighbors vs. another with just 1) can dominate the learning process. The node with 10 neighbors would aggregate far more information than the one with 1, leading to imbalance and unstable learning. Normalization ensures that each node's contribution is appropriately scaled, so the network learns from the graph structure rather than being skewed by uneven data distribution. In GCNs it's common to use symmetric normalization:

$$
\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}
$$

We do this because matrix multiplication is fundamentally asymmetrical. The rows of the first matrix get multiplied by the columns of the second matrix. If we only multiplied by the inverse of our degree matrix once, either before or after the adjacency matrix, we would asymmetrically be normalizing either rows or columns. By multiplying in this way, we are scaling down our adjacency values without a preference for rows or columns before using that to aggregate values in the hidden matrix.

The idea is to normalize each node's aggregated features by the square root of its degree (the number of neighbors, including itself for self-loops). This helps to ensure that nodes with different degrees contribute equally during aggregation.

Once we have normalized the adjacency matrix by the degree matrix, we can then multiply it by $H$ to do message passing, then we can multiply the result of that through the neural network by multiplying it by $W$ , and then through the activation function.

In [9]:
import torch_geometric.nn as gnn

class GCNLayer(nn.Module):
    """
    Implements one customizable GCN layer.
    Input -> GCNConv -> ReLU -> Dropout -> Output
    """

    def __init__(self, input_dim, output_dim, activation=None, dropout=0.0, my_gcn=False) -> None:
        super(GCNLayer, self).__init__()
        # GCN module
        if my_gcn:
            self.gcn_layer= GCNConv(input_dim, output_dim)
        else:
            self.gcn_layer= gnn.GCNConv(input_dim, output_dim)
        # Activation function
        self.activ= activation
        # Dropout layer for regularization
        self.dropout= nn.Dropout(p=dropout) if dropout> 0.0 else None


    def forward(self, data):
        x, edge_index= data

        x= self.gcn_layer(x, edge_index)
        if self.activ is not None:
            x= self.activ(x)
        if self.dropout is not None:
            x= self.dropout(x)

        return x, edge_index


In [10]:
class GCN(nn.Module):
    """
    Implementing a Graph Convolutional Network.
    nn.ReLU(inplace=True) is the default activation function.
    """

    def __init__(self, input_dim, hidden_dim=[16,], output_dim=1, activation=None,
                 dropout=0.0, my_gcn=False) -> None:
        super(GCN, self).__init__()
        # Activation function -- ReLU is the default
        activation= nn.ReLU(inplace=True) if activation is None else activation

        if isinstance(hidden_dim, int):
            hidden_dim= [hidden_dim]
        n_hidden_layers= len(hidden_dim)

        if n_hidden_layers== 0:
            raise Exception('hidden_dim cannot be an empty list')

        self.gcn_in= GCNLayer(input_dim, hidden_dim[0], activation, dropout, my_gcn)

        if n_hidden_layers> 1:
            self.gcn_hn= nn.Sequential(*[
                GCNLayer(hidden_dim[i], hidden_dim[i+1], activation, dropout, my_gcn)
                for i in range(n_hidden_layers - 1)
            ])
        else:
            self.gcn_hn= None

        self.gcn_out= GCNLayer(hidden_dim[-1], output_dim, my_gcn=my_gcn)


    def forward(self, x, edge_index):
        data= (x, edge_index)  # pack x and edge_index into a tuple

        data= self.gcn_in(data)
        if self.gcn_hn is not None:
            data= self.gcn_hn(data)  # nn.Sequential forwards only one element
        out, _= self.gcn_out(data)

        return F.log_softmax(out, dim=1)


In [11]:
model= GCN(input_dim=num_inputs, hidden_dim=[32,], output_dim=num_labels,
           dropout=0.1).to(device)

total_params= sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of parameters: {total_params}')

Number of parameters: 46119


In [12]:
model= GCN(input_dim=num_inputs, hidden_dim=[32,], output_dim=num_labels,
           dropout=0.1, my_gcn=True).to(device)

total_params= sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of parameters: {total_params}')

Number of parameters: 46119


# Training procedure

Traditional neural networks can be efficiently batched during training. For graph neural networks, it's harder to batch the data because nodes have different neighbors, resulting in potentially uneven mini-batches. Efficient sampling techniques (like GraphSAGE) or mini-batch training are necessary for scalability.

In [13]:
import copy

# training procedure - we train 10 times and calculate the average accuracy and standard deviation
def supervised_training(model_config, learning_rate=1e-3, epochs=500, eval_interval=50,
                        batches=True, batch_size=128, verbose=False):

    model_class= model_config.model_class
    input_dim= model_config.input_dim
    hidden_dim= model_config.hidden_dim
    output_dim= model_config.output_dim
    dropout= model_config.dropout

    if batches:
        epoch_size= math.floor(Xtr.shape[0]/ batch_size)
    else:
        batch_size= Xtr.shape[0]
        epoch_size= 1

    results= []
    best_test_acc= 0.0

    for i in tqdm(range(10)):
        if verbose: print(f'Training {model_class.__name__} iteration {i+1}')

        # create a fresh model for training
        if model_class== MLP:
            model_tr= MLP(input_dim, hidden_dim, output_dim, layer_norm=model_config.layer_norm,
                          dropout=dropout).to(device)
        elif model_class== GCN:
            my_gcn= True if model_config.gcn_type=='my_gcn' else False
            model_tr= GCN(input_dim, hidden_dim, output_dim, dropout=dropout, my_gcn=my_gcn).to(device)
        elif model_class== GAT:
            model_tr= GAT(input_dim, hidden_dim, output_dim, heads=model_config.heads,
                          dropout=dropout).to(device)

        # create a PyTorch optimizer
        optimizer= torch.optim.AdamW(model_tr.parameters(), lr=learning_rate, weight_decay=5e-4)

        # loss function
        class_weights= torch.bincount(data.y) / len(data.y)
        loss_fn= nn.CrossEntropyLoss(weight=1/class_weights).to(device)

        # --- training loop ---
        for epoch in range(epochs):
            # iterating over all batches
            for i in range(epoch_size):
                # --- minibatch construction ---
                Xb= Xtr[(i * batch_size):((i+1) * batch_size)]
                Yb= Ytr[(i * batch_size):((i+1) * batch_size)]

                # --- forward pass ---
                if isinstance(model_tr, MLP):
                    y_pred= model_tr(Xb)
                else:
                    y_pred= model_tr(data.x, data.edge_index)[data.train_mask]
                tr_loss= loss_fn(y_pred, Yb)

                # --- backward pass ---
                model_tr.train(True)
                optimizer.zero_grad()
                tr_loss.backward()

                # --- update ---
                optimizer.step()

            # --- track stats ---
            if epoch% eval_interval== 0:
                model_tr.eval()
                with torch.no_grad():
                    if isinstance(model_tr, MLP):
                        y_pred= model_tr(Xdev)
                    else:
                        y_pred= model_tr(data.x, data.edge_index)[data.val_mask]

                    val_loss= loss_fn(y_pred, Ydev)
                    val_acc= (y_pred.argmax(dim=1)== Ydev).sum().item()/ Ydev.shape[0]
                    if verbose:
                        print(f'Epoch {epoch} | Training Loss: {tr_loss.item():.4f} | Validation Loss: {val_loss.item():.4f} | Validation Acc: {val_acc:>5.2f}')

        # --- final evaluation on the test set ---
        model_tr.eval()
        with torch.no_grad():
            if isinstance(model_tr, MLP):
                y_pred= model_tr(Xte)
            else:
                y_pred= model_tr(data.x, data.edge_index)[data.test_mask]

            test_loss= loss_fn(y_pred, Yte)
            test_acc= (y_pred.argmax(dim=1)== Yte).sum().item()/ Yte.shape[0]
            if best_test_acc< test_acc:
                best_model= copy.deepcopy(model_tr)
            del model_tr

            if verbose: print(f'{model_class.__name__} Test Loss: {test_loss.item():.2f} | Test Acc: {test_acc:>5.2f}')
            results.append([val_acc, test_acc])

    return best_model, torch.tensor(results)


In [14]:
# --- MLP --- print average on test set and standard deviation
@dataclass
class MLPConfig:
    model_class= MLP
    input_dim= num_inputs
    hidden_dim= [32,]
    output_dim= num_labels
    layer_norm= True
    dropout= 0.1

model, results= supervised_training(MLPConfig, learning_rate=0.01, epochs=1000, eval_interval=100)
print(f'{model.__class__.__name__} - Test Accuracy: {100*results[:,1].mean():.2f} ± {100*results[:,1].std():.2f}')

100%|██████████| 10/10 [03:11<00:00, 19.18s/it]

MLP - Test Accuracy: 72.50 ± 1.21


The actual training is relatively simple. We pass our data into the model, get some output, and compare that output to our known output via cross-entropy loss. Under the hood, the hidden states of all nodes are being affected by message-passing neural networks to ultimately create a vector of 7 numbers, one for each possible class. If the biggest number in that vector is at index 0, then that vector predicts class 0. If the biggest number is in index 1, then the vector predicts class 1.

In [15]:
# --- GCN --- print average on test set and standard deviation
@dataclass
class GCNConfig:
    model_class= GCN
    gcn_type= 'gnn'
    input_dim= num_inputs
    hidden_dim= [32,]
    output_dim= num_labels
    dropout= 0.1

model, results= supervised_training(GCNConfig, learning_rate=0.01, epochs=1000, eval_interval=100,
                                    batches=False)
print(f'{model.__class__.__name__} - Test Accuracy: {100*results[:,1].mean():.2f} ± {100*results[:,1].std():.2f}')

100%|██████████| 10/10 [00:33<00:00,  3.32s/it]

GCN - Test Accuracy: 84.36 ± 0.58


In [16]:
# --- GCN --- print average on test set and standard deviation
@dataclass
class GCNConfig:
    model_class= GCN
    gcn_type= 'my_gcn'  # --- using the GCNConv module implemented from scratch ---
    input_dim= num_inputs
    hidden_dim= [32,]
    output_dim= num_labels
    dropout= 0.1

model, results= supervised_training(GCNConfig, learning_rate=0.01, epochs=1000, eval_interval=100,
                                    batches=False)
print(f'{model.__class__.__name__} - Test Accuracy: {100*results[:,1].mean():.2f} ± {100*results[:,1].std():.2f}')

100%|██████████| 10/10 [06:49<00:00, 40.94s/it]

GCN - Test Accuracy: 83.94 ± 0.46


The graph structure should really make a difference for the problem we are trying to solve. The structure should be meaningful for the prediction task at hand. Testing is important here. We can try to formulate the graph in different ways to see if one way of formulating works better than another one.

Training a graph neural network takes more time than training a normal neural network. So if the results improve only a little bit and training time is important, the normal neural network can be the best choice. Also, the effectiveness among types of graph neural networks (GCN, GAT, GraphSAGE) can vary greatly based on the problem.

Just like in standard neural networks, transfer learning (pre-training a GNN on a large dataset and fine-tuning on the target dataset) can be effective for GNNs. Checking for available pre-trained models for our task can be valuable.

As we've seen, simply adding graph information to a basic neural network can dramatically boost performance, as was the case when we moved from a normal neural network to a GCN for the Cora dataset. By aggregating information from neighboring nodes, GCNs can provide a richer representation of the data, leading to more accurate predictions. But, it's crucial to remember that GNNs aren't a magic bullet for every problem. The graph structure must be truly meaningful to the prediction task, and the increase in training complexity might not always justify the performance boost, especially when training time is critical.

In [ ]:
# https://towardsdatascience.com/graph-neural-networks-part-1-graph-convolutional-networks-explained-9c6aaa8a406e
# https://medium.com/intuitively-and-exhaustively-explained/graph-convolutional-networks-intuitively-and-exhaustively-explained-2f9afae0b040

# Graph Attention Network - GAT

GCNs treat all neighbors equally. For GATs, this is different. GATs allow the model to learn different importance (attention) scores for different neighbors. They aggregate neighbor information by using attention mechanisms (this might ring a bell because these mechanisms are also used in transformers).

In the GCN, we only looked at the degree of the nodes. GATs on the other hand, also take the feature values into account to assign attention scores to different neighbors. So instead of treating all neighbors equally, an attention mechanism is introduced that assigns varying levels of importance to different neighbors. This allows the network to focus on the most relevant parts of the graph structure, essentially learning "where to look" when making predictions.

**Computing Attention Scores:** For each node, we calculate an attention score for every neighboring node. This score is a measure of how important a specific neighbor's features are when updating the current node's features (https://arxiv.org/pdf/1710.10903). The score is learned during training, so the model decides which nodes matter most for each task (https://arxiv.org/abs/2105.14491), most of the time this method is more effective.

Just like transformers, GATs often use multi-head attention to improve their performance. Multi-head attention refers to running several separate attention mechanisms, or heads, in parallel. Each of these heads independently computes attention scores for the neighbors of a node, learning to focus on different aspects of the graph structure or node features. After these heads process the graph, their outputs are either concatenated or averaged to form the final node representation. So one of the key reasons of using multiple heads instead of one is to learn diverse patterns, because each attention head has its own learnable parameters and can learn to focus on different parts of the neighborhood. Another reason is that it stabilizes the training process. We can compare it with an ensemble, other heads can compensate for a "noisy head".

In [20]:
from torch_geometric.nn import GATv2Conv

class GATLayer(nn.Module):
    """
    Implements one customizable GAT layer.
    Input -> Dropout -> GATConv -> ELU -> Output
    """

    def __init__(self, input_dim, output_dim, heads, concat=True, activation=None,
                 dropout=0.0) -> None:
        super(GATLayer, self).__init__()
        # Dropout layer for regularization
        self.dropout= nn.Dropout(p=dropout) if dropout> 0.0 else None
        # GAT module
        self.gat_layer= GATv2Conv(input_dim, output_dim, heads=heads, concat=concat)
        # Activation function
        self.activ= activation


    def forward(self, data):
        x, edge_index= data

        if self.dropout is not None:
            x= self.dropout(x)
        x= self.gat_layer(x, edge_index)
        if self.activ is not None:
            x= self.activ(x)

        return x, edge_index



class GAT(nn.Module):
    """
    Implements a Graph Attention Network.
    nn.ELU(inplace=True) is the default activation function.
    """

    def __init__(self, input_dim, hidden_dim=[16,], output_dim=1, heads=8,
                 activation=None, dropout=0.0) -> None:
        super(GAT, self).__init__()
        # Activation function -- ELU is the default
        activation= nn.ELU(inplace=True) if activation is None else activation

        if isinstance(hidden_dim, int):
            hidden_dim= [hidden_dim]
        n_hidden_layers= len(hidden_dim)

        if n_hidden_layers== 0:
            raise Exception('hidden_dim cannot be an empty list')

        self.gat_in= GATLayer(input_dim, hidden_dim[0], heads, concat=True,
                              activation=activation, dropout=dropout)

        if n_hidden_layers> 1:
            self.gat_hn= nn.Sequential(*[
                GATLayer((d * heads), hidden_dim[i+1], heads, concat=True,
                         activation=activation, dropout=dropout)
                for i, d in enumerate(hidden_dim[:-1])
            ])
        else:
            self.gat_hn= None
        # for the last GAT layer we use concat=False to average the outputs of the heads
        self.gat_out= GATLayer((hidden_dim[-1] * heads), output_dim, heads, concat=False,
                               activation=None, dropout=dropout)


    def forward(self, x, edge_index):
        data= (x, edge_index)  # pack x and edge_index into a tuple

        data= self.gat_in(data)
        if self.gat_hn is not None:
            data= self.gat_hn(data) # nn.Sequential forwards only one element
        out, _= self.gat_out(data)

        return F.log_softmax(out, dim=1)


Each attention head computes its own set of attention scores and new node features independently. For $N$ heads, and a given node $i$, we'll end up with $N$ different sets of transformed features. Next up, all outputs are concatenated (stacked) or averaged. Concatenation is more common because it increases the model's expressiveness, but on the other hand the output dimension will be larger. Averaging helps to smooth out the differences between the heads. A general rule is to use concatenation when it's a hidden layer in the network and averaging when it's the last layer. When all attention heads are combined, we hope to get a comprehensive view of the graph, because the different heads have different perspectives on the relationships in the graph.

In [21]:
model= GAT(input_dim=num_inputs, hidden_dim=[32,], output_dim=num_labels,
           dropout=0.1).to(device)

total_params= sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of parameters: {total_params}')

Number of parameters: 763567


In [22]:
# print average on test set and standard deviation
@dataclass
class GATConfig:
    model_class= GAT
    input_dim= num_inputs
    hidden_dim= [32,]
    output_dim= num_labels
    heads= 8
    dropout= 0.1

model, results= supervised_training(GATConfig, learning_rate=0.01, epochs=1000, eval_interval=100,
                                    batches=False)
print(f'{model.__class__.__name__} - Test Accuracy: {100*results[:,1].mean():.2f} ± {100*results[:,1].std():.2f}')

100%|██████████| 10/10 [01:20<00:00,  8.07s/it]

GAT - Test Accuracy: 85.96 ± 0.94


The GAT model takes a bit longer than the GCN... The attention mechanism in GATs adds additional complexity to the model, both in terms of computation and the number of parameters. This makes GATs more resource-intensive and slower to train than GCNs.

Multi-head attention helps stabilize training, but there is still a risk of overfitting, especially when using many attention heads or deep GAT architectures. Using techniques like dropout and early stopping can help to mitigate this.

Many steps in finetuning GNNs are similar to traditional neural networks: testing different values for the hyperparameters and preventing overfitting with early stopping. For example with GATs we need to tune the number of attention heads. Small changes to node and edge features can have an impact on GNN performance, so it might help to experiment with different feature combinations or to create new features. Augmenting data can improve generalization. We can do this by adding noise to edges, randomly dropping nodes, or by performing subgraph sampling.

In [ ]:
# https://towardsdatascience.com/graph-neural-networks-part-2-graph-attention-networks-vs-gcns-029efd7a1d92